<a href="https://colab.research.google.com/github/Musa333333/business-assistant/blob/main/accurate_response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:

from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('HF-TOKEN'))  # Use your HF Token

In [3]:

from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=userdata.get('HF-TOKEN'),
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [24]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,


)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [25]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a business expert with advanced knowledge in Accounting, Banks and Banking, Business, Business Communication, Career management, E-Business, Economics, Entrepreneurship, Finance, Financial Planning, Industries and Professions, Investments, Management and Leadership, Marketing and Sales, Real Estate, Stock Trading, Tax, Small Business, and Entrepreneurship.
Please answer the following Business-related question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [26]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    inputs = examples["prompt"]
    cots = examples["thinking"]
    outputs = examples["completion"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }


In [27]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/fine_tuning_data5.jsonl", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [28]:
dataset["text"][0]

'Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a business expert with advanced knowledge in Accounting, Banks and Banking, Business, Business Communication, Career management, E-Business, Economics, Entrepreneurship, Finance, Financial Planning, Industries and Professions, Investments, Management and Leadership, Marketing and Sales, Real Estate, Stock Trading, Tax, Small Business, and Entrepreneurship.\nPlease answer the following Business-related question.\n\n### Question:\nlack of visibility into financial performance across different departments\n\n### Response:\n<think>\naccounting\n</think>\nimplement a financial performance dashboard with department-specific data and analytics<｜end▁of▁sentence｜>'

In [29]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs = 2, warmup_ratio = 0.1,
        warmup_steps=5,
        max_steps=200,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to = "none", # Use this for WandB etc
    ),
)



Map:   0%|          | 0/894 [00:00<?, ? examples/s]

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/894 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/894 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/894 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/894 [00:00<?, ? examples/s]

In [30]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 894 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,0.931800
20,0.851600
30,0.949400
40,0.914500
50,0.937400
60,0.705800
70,0.874300
80,0.911500
90,0.753400
100,0.917000


In [31]:
prompt_style ="""Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a business expert with advanced knowledge in Accounting, Banks and Banking, Business, Business Communication, Career management, E-Business, Economics, Entrepreneurship, Finance, Financial Planning, Industries and Professions, Investments, Management and Leadership, Marketing and Sales, Real Estate, Stock Trading, Tax, Small Business, and Entrepreneurship.
Please answer the following Business-related question.


### Question:
{}

### Response:
<think>{}"""

In [39]:
from langchain.output_parsers import RegexParser

def is_valid_category(query):
    """Check if the query falls within the business categories."""
    business_keywords = [
        "accounting", "banking", "business", "career", "e-business", "economics",
        "entrepreneurship", "finance", "investments", "management", "marketing",
        "real estate", "stock trading", "tax", "small business"
    ]
    return any(keyword in query.lower() for keyword in business_keywords)

question = "tell about B2B, B2C business?"

if is_valid_category(question):
    inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask, max_new_tokens=1200, use_cache=True)
    response = tokenizer.batch_decode(outputs)
    print(response[0].split("### Response:")[1])
else:
    print("I'm specialized in business-related topics. Please ask a business-related question.")



<think>
Okay, so I need to explain what B2B and B2C businesses are. Let me start by recalling what I know. B2B stands for Business-to-Business, which means selling products or services directly to other businesses. For example, a company that supplies office equipment to other businesses would be a B2B vendor. On the other hand, B2C stands for Business-to-Consumer, which involves selling directly to individual customers. A bakery selling cupcakes to the public is a classic B2C example.

I should explain the key differences between the two models. B2B often involves larger sales volumes and more complex contracts, while B2C can be more varied, including everything from small mom-and-pop shops to large retail chains. I should also mention why a business might choose one over the other. Maybe a company started as B2B but found a niche in a specific consumer market and switched to B2C to expand their customer base. Conversely, a business might move from B2C to B2B if they want to target o

In [34]:
question = "today is sunday to tell me yesterday"

FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so I need to figure out how to respond to the question "today is sunday to tell me yesterday." Hmm, that's a bit confusing at first glance. Let me break it down.

The user is asking for help with a business-related question, but the actual question seems to be about dates and time. Maybe they're trying to clarify a date or time reference. The user mentioned "today is sunday to tell me yesterday," which I'm interpreting as wanting to know what happened yesterday if today is Sunday.

So, if today is Sunday, yesterday would be Saturday. The user might be looking for information or context about an event that occurred on Saturday. Perhaps they need to reference this for a report, presentation, or discussion.

I should ask for clarification to better assist them. Maybe they have a specific business-related task or decision that requires knowing the date or event from yesterday. By seeking more details, I can provide a more accurate and helpful response.
</think>

If today is 

In [40]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

In [41]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define paths
model_save_path = "./business_solution_model"

# Save the model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


('./business_solution_model/tokenizer_config.json',
 './business_solution_model/special_tokens_map.json',
 './business_solution_model/tokenizer.json')

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: fineGr

In [42]:

from huggingface_hub import create_repo, upload_folder

# Define the model path and repository name
repo_name = "Business-assistant-thinking"  # Removed trailing space
model_path = "/content/business_solution_model"  # Replace with the path to your saved model folder

# Check if the repository exists, and skip creation if it does
try:
    create_repo(repo_name, private=False, exist_ok=True) # Use exist_ok=True to skip if it exists
except Exception as e:
    print(f"Warning: Repository creation failed with error: {e}")
    print("Continuing with upload, assuming the repository already exists.")

# Upload the model folder to Hugging Face
upload_folder(
    folder_path=model_path,
    repo_id=f"Musabjee/{repo_name}"  # Replace with your Hugging Face username and repo name
)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Musabjee/Business-assistant-thinking/commit/bcbc3af1ee31f4be4c503fc8fadcc96d8e4d26f5', commit_message='Upload folder using huggingface_hub', commit_description='', oid='bcbc3af1ee31f4be4c503fc8fadcc96d8e4d26f5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Musabjee/Business-assistant-thinking', endpoint='https://huggingface.co', repo_type='model', repo_id='Musabjee/Business-assistant-thinking'), pr_revision=None, pr_num=None)

In [43]:
import shutil

# Create a zip file of the model folder
shutil.make_archive("/content/business_solution_model", 'zip', "/content/business_solution_model")


'/content/business_solution_model.zip'

In [45]:

from google.colab import files

# Download the zip file
files.download('/content/business_solution_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>